In [4]:
import os
cwd = os.getcwd()
print(cwd)
os.chdir('/s/chopin/d/proj/ramfis-aida/coreference_and_annotations')
print(os.getcwd())

/s/chopin/d/proj/ramfis-aida/coreference_and_annotations
/s/chopin/d/proj/ramfis-aida/coreference_and_annotations


In [172]:
import torch
import torch.nn.functional as F
from transformers import LongformerModel 
# from longformer.longformer import Longformer, LongformerConfig
# from longformer.sliding_chunks import pad_to_window_size
from transformers import RobertaTokenizer
from transformers import *
import zipfile
from bs4 import BeautifulSoup as bs
from collections import OrderedDict
import pickle
import copy
import glob
import os.path
import zipfile
from bs4 import BeautifulSoup as bs
from collections import OrderedDict
import pickle
import pandas as pd
import torch 
import pyhocon
import torch.nn as nn
import torch
import torch.nn.functional as F
from transformers import *
import numpy as np

In [111]:

from parsing.parse_ldc import extract_mentions
import argparse
from collections import defaultdict
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import connected_components
from evaluations.eval import *
import os
import json
from tqdm import tqdm

/s/chopin/d/proj/ramfis-aida/coreference_and_annotations/coreference
/s/chopin/d/proj/ramfis-aida/coreference_and_annotations


In [9]:
#mentions_csv_path = '/s/chopin/d/proj/ramfis-aida/New_codescripts_embeddings/mentions_csv/mentions.csv'
mentions_csv_path = '/s/chopin/d/proj/ramfis-aida/New_codescripts_embeddings/mentions_csv_ent_ldc/ent_map.csv'
#mentions_csv_path = '/s/chopin/d/proj/ramfis-aida/CDLM-main_old/gaia_mentions_info/gaia_mentions_info/eve_map.csv'

#source_dir = '/s/chopin/d/proj/ramfis-aida/New_codescripts_embeddings'
source_dir = '/s/chopin/d/proj/ramfis-aida/coreference_and_annotations/LDCsource'
ann_dir = '/s/chopin/d/proj/ramfis-aida/coreference_and_annotations/LDCann'

output_folder = '/s/chopin/d/proj/ramfis-aida/New_codescripts_embeddings'
working_folder = output_folder + '/WORKING_LDC_PHASE3_cosim/'


In [283]:
def get_mention_pair_similarity_lemma(mention_pairs, mention_map, relations, working_folder):
    """
    Generate the similarities for mention pairs

    Parameters
    ----------
    mention_pairs: list
    mention_map: dict
    relations: list
        The list of relations represented as a triple: (head, label, tail)
    working_folder: str

    Returns
    -------
    list
    """
    similarities = []

    # generate similarity using the mention text
    for pair in mention_pairs:
        men1, men2 = pair
        men_map1 = mention_map[men1]
        men_map2 = mention_map[men2]
        men_text1 = men_map1['mention_text']
        men_text2 = men_map2['mention_text']
        similarities.append(int(men_text1 == men_text2))
        #print(similarities)
        #print(len(similarities))

    return similarities

def get_mention_pair_cosinesimilarity(mention_pairs, mention_map,evt_vec_map, relations, working_folder):
    """
    Generate the cosine similarities for mention pairs using the CDLM model 

    Parameters
    ----------
    mention_pairs: list
    mention_map: dict
    evt_vec_map : dict with mention IDs as keys and CDLM Embeddings as values
    relations: list
        The list of relations represented as a triple: (head, label, tail)
    working_folder: str

    Returns
    -------
    list
    """
    similarities = []
    sims_list = []
    cos_s = torch.nn.CosineSimilarity()
      # generate similarity using the mention text
    for pair in mention_pairs:
        men1, men2 = pair
        men_vec1 = torch.tensor(evt_vec_map[men1].reshape(1,-1)) 
        men_vec2 = torch.tensor(evt_vec_map[men2].reshape(1,-1))
        sims = cos_s(men_vec1,men_vec2).data.cpu().numpy()
        #sims[sims>0.98757007] = 1  #mean for cdlm model 
        #sims[sims<=0.98757007] =0
        #sims[sims>0.95] = 1
        #sims[sims<=0.95] =0
        #sims[sims>0.47] = 1  # mean for coref bert model 
        #sims[sims<=0.47] =0
#         sims[sims>0.39636973] = 1  # mean for bert large  model 
#         sims[sims<=0.39636973] =0
        
#         sims[sims>0.6] = 1  #  
#         sims[sims<=0.6] =0
        
        sims[sims>0.7] = 1  #  
        sims[sims<=0.7] =0
        sims_list.append([pair, sims])
        similarities.append(sims)

    return similarities, sims_list

def cluster_cc(affinity_matrix, threshold=0.8):
    """
    Find connected components using the affinity matrix and threshold -> adjacency matrix
    Parameters
    ----------
    affinity_matrix: np.array
    threshold: float

    Returns
    -------
    list, np.array
    """
    adjacency_matrix = csr_matrix(affinity_matrix > threshold)
    clusters, labels = connected_components(adjacency_matrix, return_labels=True, directed=False)
    return clusters, labels


def run_coreference(ann_dir, source_dir, working_folder, men_type='evt'):
    """
    Run the coreference resolution pipeline on LDC annotations in the following steps:
        1) Read and parse the annotations + source documents
        2) Generate pairs of mentions from the same topic
        3) Generate similarities for the pairs and create a similarity matrix
        4) Agglomeratively cluster using the similarity matrix
        5) Evaluate the clusters against the gold standard clusters (BCUB, MUC, CEAF, CONNL)

    Parameters
    ----------
    ann_dir: str
    source_dir: str
    working_folder: str
    men_type: str

    Returns
    -------
    None
    """
    # create working folder
    if not os.path.exists(working_folder):
        os.makedirs(working_folder)

    # extract the mention maps
    eve_mention_map, ent_mention_map, relations, doc_sent_map = extract_mentions(ann_dir, source_dir, working_folder)

    # which coreference mention map
    if men_type == 'evt':
        curr_mention_map = eve_mention_map
    else:
        curr_mention_map = ent_mention_map

    # create a single dict for all mentions
    all_mention_map = {**eve_mention_map, **ent_mention_map}

    # sort event mentions and make men to ind map
    curr_mentions = sorted(list(curr_mention_map.keys()), key=lambda x: curr_mention_map[x]['m_id'])
    curr_men_to_ind = {eve: i for i, eve in enumerate(curr_mentions)}

    # generate gold clusters key file
    curr_gold_cluster_map = [(men, all_mention_map[men]['gold_cluster']) for men in curr_mentions]
    gold_key_file = working_folder + f'/{men_type}_gold.keyfile'
    generate_key_file(curr_gold_cluster_map, men_type, working_folder, gold_key_file)

    # group mentions by topic
    topic_mention_dict = defaultdict(list)
    for men_id, coref_map in curr_mention_map.items():
        topic = coref_map['topic']
        topic_mention_dict[topic].append(men_id)

    # generate mention-pairs
    mention_pairs = []
    for mentions in topic_mention_dict.values():
        list_mentions = list(mentions)
        for i in range(len(list_mentions)):
            for j in range(i+1):
                if i != j:
                    mention_pairs.append((list_mentions[i], list_mentions[j]))

    # get the similarities of the mention-pairs
    similarities = get_mention_pair_similarity_lemma(mention_pairs, all_mention_map, relations, working_folder)

    # get indices
    mention_ind_pairs = [(curr_men_to_ind[mp[0]], curr_men_to_ind[mp[1]]) for mp in mention_pairs]
    row, col = zip(*mention_ind_pairs)

    # create similarity matrix from the similarities
    n = len(curr_mentions)
    similarity_matrix = csr_matrix((similarities, (row, col)), shape=(n, n)).toarray()

    # clustering algorithm and mention cluster map
    clusters, labels = connected_components(similarity_matrix)
    system_mention_cluster_map = [(men, clus) for men, clus in zip(curr_mentions, labels)]

    # generate system key file
    system_key_file = working_folder + f'/{men_type}_system.keyfile'
    generate_key_file(system_mention_cluster_map, men_type, working_folder, system_key_file)

    # evaluate
    generate_results(gold_key_file, system_key_file)

In [14]:
men_type = 'evt'

eve_mention_map, ent_mention_map, relations, doc_sent_map  = extract_mentions(ann_dir, source_dir, working_folder)

# # which coreference mention map
# if men_type == 'evt':
#     curr_mention_map = eve_mention_map
# else:
#     curr_mention_map = ent_mention_map

# # create a single dict for all mentions
# all_mention_map = {**eve_mention_map, **ent_mention_map}

# # sort event mentions and make men to ind map
# curr_mentions = sorted(list(curr_mention_map.keys()), key=lambda x: curr_mention_map[x]['m_id'])
# curr_men_to_ind = {eve: i for i, eve in enumerate(curr_mentions)}

# # generate gold clusters key file
# curr_gold_cluster_map = [(men, all_mention_map[men]['gold_cluster']) for men in curr_mentions]
# gold_key_file = working_folder + f'/{men_type}_gold.keyfile'
# generate_key_file(curr_gold_cluster_map, men_type, working_folder, gold_key_file)

# # group mentions by topic
# topic_mention_dict = defaultdict(list)
# for men_id, coref_map in curr_mention_map.items():
#     topic = coref_map['topic']
#     topic_mention_dict[topic].append(men_id)

# # generate mention-pairs
# mention_pairs = []
# for mentions in topic_mention_dict.values():
#     list_mentions = list(mentions)
#     for i in range(len(list_mentions)):
#         for j in range(i+1):
#             if i != j:
#                 mention_pairs.append((list_mentions[i], list_mentions[j]))

# # get the similarities of the mention-pairs
# similarities = get_mention_pair_similarity_lemma(mention_pairs, all_mention_map, relations, working_folder)

/s/chopin/d/proj/ramfis-aida/coreference_and_annotations/parsing/parse_ldc.py:114: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 114 of the file /s/chopin/d/proj/ramfis-aida/coreference_and_annotations/parsing/parse_ldc.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  file_bs = bs(ltf_xml, from_encoding='utf-8')


In [11]:
eve_mention_map_file = working_folder + '/evt_mention_map_bertdoc.pkl'

In [12]:
vector_map_evt_bertdoc = pickle.load(open(eve_mention_map_file, 'rb'))

In [15]:
vector_map_evt_bertdoc['VMIC0015MVH.001906']

{'m_id': 704,
 'mention_id': 'VMIC0015MVH.001906',
 'doc_id': 'HC0008OSY',
 'start_char': 3630,
 'end_char': 3638,
 'gold_cluster': 'NILV03220',
 'topic': 'E101',
 'lang_id': 'eng',
 'mention_text': 'a warning',
 'sentence_start_char': 3607,
 'sentence': 'The murder is meant as a warning for all those who dare to oppose Kiev government, which can do anything against the opponents, Tsarev believes.',
 'bert_sentence': 'The murder is meant as  <m> a warning </m>  for all those who dare to oppose Kiev government, which can do anything against the opponents, Tsarev believes.',
 'bert_doc': 'Series of ‘bizarre suicides’ &amp;amp; murders: Former Ukrainian MP shot dead in Kiev Oleg Kalashnikov (image by http://glavpost.com) \u200bA former Ukrainian MP and active anti-Maidan activist, Oleg Kalashnikov, has been killed in his flat in Kiev. His killing is the latest in a series of odd deaths plaguing former government officials and ex-President Yanukovichs party members. The 52-year-old was fou

In [79]:
vector_map_path = working_folder + '/cdlm_evt_vector_map_events_LDCPhase3.pkl'
#men_ids, bert_sentences = zip(*[(men_id, men['bert_sentence']) for men_id, men in eve_mention_map.items()])
men_ids, bert_sentences = zip(*[(men_id, men['bert_doc']) for men_id, men in vector_map_evt_bertdoc.items()])

In [17]:
tokenizer_bert_coref = AutoTokenizer.from_pretrained('/s/chopin/d/proj/ramfis-aida/coreference_and_annotations/coreference/coref_bert_base/coref_token')
model_coref = AutoModel.from_pretrained('/s/chopin/d/proj/ramfis-aida/coreference_and_annotations/coreference/coref_bert_base')

loading file /s/chopin/d/proj/ramfis-aida/coreference_and_annotations/coreference/coref_bert_base/coref_token/vocab.txt
loading file /s/chopin/d/proj/ramfis-aida/coreference_and_annotations/coreference/coref_bert_base/coref_token/tokenizer.json
loading file /s/chopin/d/proj/ramfis-aida/coreference_and_annotations/coreference/coref_bert_base/coref_token/added_tokens.json
loading file /s/chopin/d/proj/ramfis-aida/coreference_and_annotations/coreference/coref_bert_base/coref_token/special_tokens_map.json
loading file /s/chopin/d/proj/ramfis-aida/coreference_and_annotations/coreference/coref_bert_base/coref_token/tokenizer_config.json
loading configuration file /s/chopin/d/proj/ramfis-aida/coreference_and_annotations/coreference/coref_bert_base/config.json
Model config BertConfig {
  "_name_or_path": "/s/chopin/d/proj/ramfis-aida/coreference_and_annotations/coreference/coref_bert_base",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout":

In [219]:
tokenizer_bert_large = BertTokenizer.from_pretrained('/s/chopin/d/proj/ramfis-aida/coreference_and_annotations/coreference/bert_large')
model_bert_large = BertModel.from_pretrained("/s/chopin/d/proj/ramfis-aida/coreference_and_annotations/coreference/bert_large")

loading file /s/chopin/d/proj/ramfis-aida/coreference_and_annotations/coreference/bert_large/vocab.txt
loading file /s/chopin/d/proj/ramfis-aida/coreference_and_annotations/coreference/bert_large/added_tokens.json
loading file /s/chopin/d/proj/ramfis-aida/coreference_and_annotations/coreference/bert_large/special_tokens_map.json
loading file /s/chopin/d/proj/ramfis-aida/coreference_and_annotations/coreference/bert_large/tokenizer_config.json
Adding <m> to the vocabulary
Adding </m> to the vocabulary
loading configuration file /s/chopin/d/proj/ramfis-aida/coreference_and_annotations/coreference/bert_large/config.json
Model config BertConfig {
  "_name_or_path": "bert-large-uncased",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
 

In [220]:
start_id = tokenizer_bert_coref.encode('<m>', add_special_tokens=False)[0]
end_id = tokenizer_bert_coref.encode('</m>', add_special_tokens=False)[0]
start_id,end_id

(28996, 28997)

In [222]:
#model_coref.config 
model_bert_large.config

BertConfig {
  "_name_or_path": "/s/chopin/d/proj/ramfis-aida/coreference_and_annotations/coreference/bert_large",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30524
}

In [224]:
coref_model_path ='/s/chopin/d/proj/ramfis-aida/coreference_and_annotations/coreference/coref_bert_base'
bert_large_path = "/s/chopin/d/proj/ramfis-aida/coreference_and_annotations/coreference/bert_large"
bert_large_tokenizer_path = '/s/chopin/d/proj/ramfis-aida/coreference_and_annotations/coreference/bert_large'

In [225]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        nn.init.uniform_(m.bias)

class FullCrossEncoder(nn.Module):
    def __init__(self, config, is_training=True, long=False):
        super(FullCrossEncoder, self).__init__()
        #self.segment_size = config.segment_window * 2
        #self.tokenizer = AutoTokenizer.from_pretrained('/s/chopin/d/proj/ramfis-aida/coreference_and_annotations/coreference/coref_bert_base/coref_token')
        self.tokenizer = BertTokenizer.from_pretrained(bert_large_tokenizer_path)
        


        #self.tokenizer.add_tokens(['<m>', '</m>'], special_tokens = True)
        #self.tokenizer.add_tokens(['<g>'], special_tokens = True)
        self.start_id = self.tokenizer.encode('<m>', add_special_tokens=False)[0]
        self.end_id = self.tokenizer.encode('</m>', add_special_tokens=False)[0]

        self.vals = [self.start_id, self.end_id]
        self.long = long
        if not is_training and config.pretrained_model:
            self.model = AutoModel.from_pretrained(config.cdlm_path)
        else:
            #self.model = AutoModel.from_pretrained(coref_model_path)
            self.model = BertModel.from_pretrained(bert_large_path)
            self.model.resize_token_embeddings(len(self.tokenizer))
        self.hidden_size = self.model.config.hidden_size
        if not self.long:
            self.linear = nn.Sequential(
                nn.Linear(self.hidden_size, 128),
                nn.ReLU(),
                nn.Linear(128, 1)
            )
        else:
            self.linear = nn.Sequential(
                nn.Linear(self.hidden_size*4, self.hidden_size),
                nn.ReLU(),
                nn.Linear(self.hidden_size, 128),
                nn.ReLU(),
                nn.Linear(128, 1)
            )
        self.linear.apply(init_weights)



    def forward(self, input_ids, attention_mask=None, arg1=None, arg2=None):
        output, _ = self.model(input_ids, attention_mask=attention_mask)
        if self.long:
            arg1_vec = (output*arg1.unsqueeze(-1)).sum(1)
            arg2_vec = (output*arg2.unsqueeze(-1)).sum(1)
        cls_vector = output[:, 0, :]
        if not self.long:
            scores = self.linear(cls_vector)
        else:
            scores = self.linear(torch.cat([cls_vector, arg1_vec, arg2_vec, arg1_vec*arg2_vec],dim=1))
        return scores

    def generate_rep(self, input_ids, attention_mask=None, arg1=None,):
        output, _ = self.model(input_ids, attention_mask=attention_mask)
        arg1_vec = (output*arg1.unsqueeze(-1)).sum(1)
        return arg1_vec


class FullCrossEncoderSingle(FullCrossEncoder):
    def __init__(self, config, is_training=True, long=False):
        super(FullCrossEncoderSingle, self).__init__(config, is_training=is_training, long=long)

    def forward(self, input_ids, attention_mask=None, arg1=None, arg2=None):
        output = self.model(input_ids, attention_mask=attention_mask).last_hidden_state
         
        arg1_vec = (output * arg1.unsqueeze(-1)).sum(1)
        return arg1_vec

In [149]:
cross_encoder = FullCrossEncoderSingle(model_coref.config , long=True)


loading file /s/chopin/d/proj/ramfis-aida/coreference_and_annotations/coreference/coref_bert_base/coref_token/vocab.txt
loading file /s/chopin/d/proj/ramfis-aida/coreference_and_annotations/coreference/coref_bert_base/coref_token/tokenizer.json
loading file /s/chopin/d/proj/ramfis-aida/coreference_and_annotations/coreference/coref_bert_base/coref_token/added_tokens.json
loading file /s/chopin/d/proj/ramfis-aida/coreference_and_annotations/coreference/coref_bert_base/coref_token/special_tokens_map.json
loading file /s/chopin/d/proj/ramfis-aida/coreference_and_annotations/coreference/coref_bert_base/coref_token/tokenizer_config.json
loading configuration file /s/chopin/d/proj/ramfis-aida/coreference_and_annotations/coreference/coref_bert_base/config.json
Model config BertConfig {
  "_name_or_path": "/s/chopin/d/proj/ramfis-aida/coreference_and_annotations/coreference/coref_bert_base",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout":

In [150]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Using device: cuda

NVIDIA GeForce RTX 3090
Memory Usage:
Allocated: 1.9 GB
Cached:    2.8 GB


In [151]:
 
cross_encoder = cross_encoder.to(device)
cross_encoder.model = AutoModel.from_pretrained(coref_model_path).to(device)

loading configuration file /s/chopin/d/proj/ramfis-aida/coreference_and_annotations/coreference/coref_bert_base/config.json
Model config BertConfig {
  "_name_or_path": "/s/chopin/d/proj/ramfis-aida/coreference_and_annotations/coreference/coref_bert_base",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28998
}

loading weights file /s/chopin/d/proj/ramfis-aida/coreference_and_annotations/coreference/coref_bert_base/pytorch_model.bin
All model chec

In [153]:
parallel_coref_model = torch.nn.DataParallel(cross_encoder)
parallel_coref_model.module.to(device)

FullCrossEncoderSingle(
  (model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28998, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [154]:
parallel_coref_model.eval()
parallel_coref_model.module.tokenizer

PreTrainedTokenizerFast(name_or_path='/s/chopin/d/proj/ramfis-aida/coreference_and_annotations/coreference/coref_bert_base/coref_token', vocab_size=28996, model_max_len=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]', 'additional_special_tokens': ['<m>', '</m>']})

In [57]:
#bert_sentences =bert_sentences[0:20]
parallel_coref_model.module.vals[0]
parallel_coref_model.module.vals[1]
 

28997

In [80]:
loc_mentions = []
bert_sentences
batch = [bert_sentences[x] for x in range( len(bert_sentences))]
 
bert_tokens = parallel_coref_model.module.tokenizer(batch, pad_to_max_length=True, add_special_tokens=False, truncation =False )
#bert_tokens = parallel_model.module.tokenizer(batch, padding = 'max_length', add_special_tokens=False, truncation =True )
input_ids = torch.tensor(bert_tokens['input_ids'], device=device)
attention_mask = torch.tensor(bert_tokens['attention_mask'], device=device)
m = input_ids.cpu()
k = m == parallel_coref_model.module.vals[0]
p = m == parallel_coref_model.module.vals[1]
v = (k.int() + p.int()).bool()
nz_indexes = v.nonzero()[:, 1].reshape(m.shape[0], 2)
print(nz_indexes.shape)
#print(nz_indexes[0][1])
loc_mentions.extend(nz_indexes.cpu().numpy())
print(input_ids.shape)
loc_mentions



torch.Size([847, 2])
torch.Size([847, 12006])


[array([134, 136]),
 array([69, 71]),
 array([162, 164]),
 array([491, 493]),
 array([663, 666]),
 array([362, 366]),
 array([747, 750]),
 array([748, 750]),
 array([921, 924]),
 array([963, 965]),
 array([ 999, 1001]),
 array([ 999, 1001]),
 array([1028, 1030]),
 array([1025, 1027]),
 array([1058, 1060]),
 array([1058, 1060]),
 array([1103, 1105]),
 array([1135, 1137]),
 array([1135, 1137]),
 array([1148, 1150]),
 array([1176, 1178]),
 array([1191, 1194]),
 array([1213, 1215]),
 array([1213, 1218]),
 array([243, 248]),
 array([663, 666]),
 array([560, 563]),
 array([1041, 1043]),
 array([1061, 1064]),
 array([1192, 1195]),
 array([301, 303]),
 array([40, 43]),
 array([441, 443]),
 array([277, 279]),
 array([607, 610]),
 array([623, 626]),
 array([643, 646]),
 array([440, 442]),
 array([614, 617]),
 array([631, 633]),
 array([644, 646]),
 array([502, 505]),
 array([38, 40]),
 array([28, 31]),
 array([28, 31]),
 array([63, 66]),
 array([63, 66]),
 array([133, 136]),
 array([133, 136]),


In [88]:
overflow_idx=[]
[x[1] for x in loc_mentions]
for i,j in enumerate(loc_mentions):
    if j[1]>=512:
        overflow_idx.append([i, j[1] ])
        
        
        
idx = [x[0] for x in overflow_idx]

# res=list(np.array(bert_sentences)[idx])
# len(res)

m_new  = []
for i, j in enumerate(overflow_idx):
    print(j )
    bert_sent = bert_sentences[j[0] ]
    
    #m[j[0]] = m[j[0], j[1]-4096:j[1]] 
    print("m shape",m[j[0],  j[1]-512+1:j[1]+1].shape,j[0] )
    m_new.append(m[j[0],  j[1]-512+1:j[1]+1] )
    print(len(bert_sent.split()))
    

[4, 666]
m shape torch.Size([512]) 4
740
[6, 750]
m shape torch.Size([512]) 6
740
[7, 750]
m shape torch.Size([512]) 7
740
[8, 924]
m shape torch.Size([512]) 8
740
[9, 965]
m shape torch.Size([512]) 9
740
[10, 1001]
m shape torch.Size([512]) 10
741
[11, 1001]
m shape torch.Size([512]) 11
741
[12, 1030]
m shape torch.Size([512]) 12
741
[13, 1027]
m shape torch.Size([512]) 13
740
[14, 1060]
m shape torch.Size([512]) 14
740
[15, 1060]
m shape torch.Size([512]) 15
740
[16, 1105]
m shape torch.Size([512]) 16
740
[17, 1137]
m shape torch.Size([512]) 17
740
[18, 1137]
m shape torch.Size([512]) 18
740
[19, 1150]
m shape torch.Size([512]) 19
740
[20, 1178]
m shape torch.Size([512]) 20
740
[21, 1194]
m shape torch.Size([512]) 21
740
[22, 1215]
m shape torch.Size([512]) 22
740
[23, 1218]
m shape torch.Size([512]) 23
740
[25, 666]
m shape torch.Size([512]) 25
740
[26, 563]
m shape torch.Size([512]) 26
740
[27, 1043]
m shape torch.Size([512]) 27
741
[28, 1064]
m shape torch.Size([512]) 28
741
[29, 

In [93]:
m_new_tensor = torch.stack(m_new) 
 
#torch.nn.functional.pad(m_new_tensor, pad = (0,1), mode='constant', value=1)
m_new_tensor = F.pad(m_new_tensor,pad =(0, m.shape[1]-m_new_tensor.shape[1]), value=1) 

In [94]:
m_new_tensor.shape

torch.Size([418, 12006])

In [97]:
m[idx] = m_new_tensor

In [107]:
k = m == parallel_coref_model.module.vals[0]
p = m == parallel_coref_model.module.vals[1]

v = (k.int() + p.int()).bool()
nz_indexes = v.nonzero()[:, 1].reshape(m.shape[0], 2)

q = torch.arange(m.shape[1])
q = q.repeat(m.shape[0], 1)

msk_0 = (nz_indexes[:, 0].repeat(m.shape[1], 1).transpose(0, 1)) <= q
msk_1 = (nz_indexes[:, 1].repeat(m.shape[1], 1).transpose(0, 1)) >= q

msk_0_ar = (nz_indexes[:, 0].repeat(m.shape[1], 1).transpose(0, 1)) < q
msk_1_ar = (nz_indexes[:, 1].repeat(m.shape[1], 1).transpose(0, 1)) > q

attention_mask_g = msk_0.int() * msk_1.int()

input_ids = input_ids[:, :512]
# attention_mask = attention_mask[:, :4096]
#attention_mask[:, 0] = 2
#attention_mask[attention_mask_g == 1] = 2
attention_mask = attention_mask[:, :512]
arg1 = msk_0_ar.int() * msk_1_ar.int()
arg_new1 = msk_0.int() * msk_1.int()
arg_new1 = arg_new1[:,:512]
arg1 = arg1[:, :512]
arg1 = arg1.to(device)


In [112]:
batch_size = 150
batch = [input_ids[x] for x in range(i, min(i + batch_size, len(bert_sentences)))]
for i in tqdm(range(0, len(bert_sentences), batch_size), desc="generating "):
    print(input_ids[i:i+batch_size].shape)
    
    print(i)

generating : 100%|██████████| 6/6 [00:00<00:00, 32305.29it/s]

torch.Size([150, 512])
0
torch.Size([150, 512])
150
torch.Size([150, 512])
300
torch.Size([150, 512])
450
torch.Size([150, 512])
600
torch.Size([97, 512])
750


In [155]:
all_vectors = []
batch_size = 50
with torch.no_grad():
    for i in tqdm(range(0, len(bert_sentences), batch_size), desc="generating "):
        #men_vectors = parallel_coref_model(input_ids[i:i+batch_size], attention_mask[i:i+batch_size], arg1[i:i+batch_size]).detach().cpu().numpy()
        men_vectors = parallel_coref_model(input_ids[i:i+batch_size], attention_mask[i:i+batch_size], arg1[i:i+batch_size]) 
        all_vectors.extend(men_vectors)
        

generating : 100%|██████████| 17/17 [00:03<00:00,  4.48it/s]


In [158]:
len(all_vectors)

847

In [159]:
zip_iterator = zip(men_ids, all_vectors)

In [160]:
corefbert_eve_vec_map_longdoc = dict(zip_iterator)

In [161]:
corefbert_eve_vec_map_longdoc['VMIC0015MXG.000003'] 

tensor([ 1.3762e+00,  3.9003e-01,  2.7317e-01,  1.1907e-01,  6.9881e-01,
         2.3653e-02, -6.3391e-02,  4.1318e-02,  1.9541e-01, -3.4547e-01,
         2.3975e-01, -4.5153e-01, -9.4092e-01,  1.5608e-01, -6.5492e-01,
        -1.3030e-01,  4.8318e-01, -1.0549e+00, -4.3398e-02,  1.6455e-01,
        -7.9633e-01,  2.2138e-01, -6.0868e-01,  7.2251e-01,  7.7709e-01,
        -1.1637e+00,  3.2054e-01,  6.4990e-01,  3.4371e-01, -7.3551e-01,
         1.3439e+00, -4.0253e-01, -6.9410e-02,  7.6835e-01, -9.4313e-01,
        -4.1650e-01,  7.8101e-01, -4.5216e-01, -1.0445e+00, -2.5581e-01,
         5.5291e-01, -6.0835e-01, -9.3392e-01,  1.7786e+00, -1.1582e-01,
        -9.8942e-02, -2.1288e-02,  5.5209e-01, -9.4017e-01, -5.9559e-01,
         1.7318e-01,  1.9765e-01,  1.3828e-01,  1.9807e-01, -6.4563e-02,
        -4.7534e-01, -8.1373e-01, -7.4151e-01, -4.3523e-01,  1.3022e+00,
        -1.1737e-01, -6.5041e-01, -1.1155e-01, -3.5110e-01, -2.7437e-01,
         1.2891e+00, -6.7272e-01, -2.2472e-01,  4.4

In [163]:
corefbert_eve_mention_map_longdoc_file = working_folder + '/corefbert_evt_vectorn_map_longbertdoc.pkl'
#pickle.dump(corefbert_eve_vec_map_longdoc, open(corefbert_eve_mention_map_longdoc_file, 'wb'))

In [165]:
corefbert_vector_map_evt_longbertdoc = pickle.load(open(corefbert_eve_mention_map_longdoc_file, 'rb'))

In [167]:

# # which coreference mention map
# if men_type == 'evt':
#     curr_mention_map = eve_mention_map
# else:
#     curr_mention_map = ent_mention_map

curr_mention_map = eve_mention_map 
# create a single dict for all mentions
all_mention_map = {**eve_mention_map, **ent_mention_map}

# sort event mentions and make men to ind map
curr_mentions = sorted(list(curr_mention_map.keys()), key=lambda x: curr_mention_map[x]['m_id'])
curr_men_to_ind = {eve: i for i, eve in enumerate(curr_mentions)}

# generate gold clusters key file
curr_gold_cluster_map = [(men, all_mention_map[men]['gold_cluster']) for men in curr_mentions]
gold_key_file = working_folder + f'/{men_type}_gold.keyfile'
generate_key_file(curr_gold_cluster_map, men_type, working_folder, gold_key_file)

# group mentions by topic
topic_mention_dict = defaultdict(list)
for men_id, coref_map in curr_mention_map.items():
    topic = coref_map['topic']
    topic_mention_dict[topic].append(men_id)

# generate mention-pairs
mention_pairs = []
for mentions in topic_mention_dict.values():
    list_mentions = list(mentions)
    for i in range(len(list_mentions)):
        for j in range(i+1):
            if i != j:
                mention_pairs.append((list_mentions[i], list_mentions[j]))

# get the similarities of the mention-pairs
# similarities = get_mention_pair_similarity_lemma(mention_pairs, all_mention_map, relations, working_folder)

In [169]:
len(all_mention_map)

847

In [195]:
#similarities = get_mention_pair_similarity_lemma(mention_pairs, all_mention_map, relations, working_folder)
similarities, _ = get_mention_pair_cosinesimilarity(mention_pairs, all_mention_map,corefbert_vector_map_evt_longbertdoc, relations, working_folder)

/s/chopin/d/proj/ramfis-aida/venv/lib/python3.6/site-packages/ipykernel_launcher.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/s/chopin/d/proj/ramfis-aida/venv/lib/python3.6/site-packages/ipykernel_launcher.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [196]:
similarities = np.asarray(similarities)
np.max(similarities), np.min(similarities), np.mean(similarities)

(1.0, 0.0, 0.48732683)

In [197]:
similarities

array([[1.],
       [1.],
       [1.],
       ...,
       [0.],
       [1.],
       [1.]], dtype=float32)

# Coref results with cosine sim threshold of 0.6 for coref bert 512 model 

In [188]:
# get the similarities of the mention-pairs
# similarities = get_mention_pair_similarity_lemma(mention_pairs, all_mention_map, relations, working_folder)

# get indices
mention_ind_pairs = [(curr_men_to_ind[mp[0]], curr_men_to_ind[mp[1]]) for mp in mention_pairs]
row, col = zip(*mention_ind_pairs)
# print(row, col)

# create similarity matrix from the similarities
n = len(curr_mentions)
similarities = [int(x) for x in similarities ] # to avoid ValueError: row, column, and data arrays must be 1-D
similarity_matrix = csr_matrix((similarities, (row, col)), shape=(n, n)).toarray()

# clustering algorithm and mention cluster map
clusters, labels = connected_components(similarity_matrix)
system_mention_cluster_map = [(men, clus) for men, clus in zip(curr_mentions, labels)]

# generate system key file
system_key_file = working_folder + f'/{men_type}_system.keyfile'
generate_key_file(system_mention_cluster_map, men_type, working_folder, system_key_file)

# evaluate
generate_results(gold_key_file, system_key_file)

BCUB SCORE
====== TOTALS =======
Identification of Mentions: Recall: (847 / 847) 100%	Precision: (847 / 847) 100%	F1: 100%
--------------------------------------------------------------------------
Coreference: Recall: (637.927625503153 / 847) 75.31%	Precision: (174.610442332893 / 847) 20.61%	F1: 32.37%
--------------------------------------------------------------------------

MUC SCORE
====== TOTALS =======
Identification of Mentions: Recall: (847 / 847) 100%	Precision: (847 / 847) 100%	F1: 100%
--------------------------------------------------------------------------
Coreference: Recall: (480 / 576) 83.33%	Precision: (480 / 745) 64.42%	F1: 72.67%
--------------------------------------------------------------------------



#  # Coref results with cosine sim threshold of 0.47(mean of original cosine sims) for coref bert 512 model 

In [198]:
# get the similarities of the mention-pairs
# similarities = get_mention_pair_similarity_lemma(mention_pairs, all_mention_map, relations, working_folder)

# get indices
mention_ind_pairs = [(curr_men_to_ind[mp[0]], curr_men_to_ind[mp[1]]) for mp in mention_pairs]
row, col = zip(*mention_ind_pairs)
# print(row, col)

# create similarity matrix from the similarities
n = len(curr_mentions)
similarities = [int(x) for x in similarities ] # to avoid ValueError: row, column, and data arrays must be 1-D
similarity_matrix = csr_matrix((similarities, (row, col)), shape=(n, n)).toarray()

# clustering algorithm and mention cluster map
clusters, labels = connected_components(similarity_matrix)
system_mention_cluster_map = [(men, clus) for men, clus in zip(curr_mentions, labels)]

# generate system key file
system_key_file = working_folder + f'/{men_type}_system.keyfile'
generate_key_file(system_mention_cluster_map, men_type, working_folder, system_key_file)

# evaluate
generate_results(gold_key_file, system_key_file)

BCUB SCORE
====== TOTALS =======
Identification of Mentions: Recall: (847 / 847) 100%	Precision: (847 / 847) 100%	F1: 100%
--------------------------------------------------------------------------
Coreference: Recall: (844 / 847) 99.64%	Precision: (59.5594266564067 / 847) 7.03%	F1: 13.13%
--------------------------------------------------------------------------

MUC SCORE
====== TOTALS =======
Identification of Mentions: Recall: (847 / 847) 100%	Precision: (847 / 847) 100%	F1: 100%
--------------------------------------------------------------------------
Coreference: Recall: (573 / 576) 99.47%	Precision: (573 / 843) 67.97%	F1: 80.76%
--------------------------------------------------------------------------



# coref results for CDLM model with cos sim threshold 0.95

In [199]:
eve_mention_map_longdoc_file = working_folder + '/evt_vectorn_map_longbertdoc.pkl'

In [200]:
vector_map_evt_longbertdoc = pickle.load(open(eve_mention_map_longdoc_file, 'rb'))

In [213]:
#similarities = get_mention_pair_similarity_lemma(mention_pairs, all_mention_map, relations, working_folder)
similarities, _ = get_mention_pair_cosinesimilarity(mention_pairs, all_mention_map,vector_map_evt_longbertdoc, relations, working_folder)

In [214]:
similarities = np.asarray(similarities)
np.max(similarities), np.min(similarities), np.mean(similarities)

(1.0, 0.0, 0.98890084)

In [215]:
# get the similarities of the mention-pairs
# similarities = get_mention_pair_similarity_lemma(mention_pairs, all_mention_map, relations, working_folder)

# get indices
mention_ind_pairs = [(curr_men_to_ind[mp[0]], curr_men_to_ind[mp[1]]) for mp in mention_pairs]
row, col = zip(*mention_ind_pairs)
# print(row, col)

# create similarity matrix from the similarities
n = len(curr_mentions)
similarities = [int(x) for x in similarities ] # to avoid ValueError: row, column, and data arrays must be 1-D
similarity_matrix = csr_matrix((similarities, (row, col)), shape=(n, n)).toarray()

# clustering algorithm and mention cluster map
clusters, labels = connected_components(similarity_matrix)
system_mention_cluster_map = [(men, clus) for men, clus in zip(curr_mentions, labels)]

# generate system key file
system_key_file = working_folder + f'/{men_type}_system.keyfile'
generate_key_file(system_mention_cluster_map, men_type, working_folder, system_key_file)

# evaluate
generate_results(gold_key_file, system_key_file)

BCUB SCORE
====== TOTALS =======
Identification of Mentions: Recall: (847 / 847) 100%	Precision: (847 / 847) 100%	F1: 100%
--------------------------------------------------------------------------
Coreference: Recall: (844 / 847) 99.64%	Precision: (58.5333491877758 / 847) 6.91%	F1: 12.92%
--------------------------------------------------------------------------

MUC SCORE
====== TOTALS =======
Identification of Mentions: Recall: (847 / 847) 100%	Precision: (847 / 847) 100%	F1: 100%
--------------------------------------------------------------------------
Coreference: Recall: (573 / 576) 99.47%	Precision: (573 / 844) 67.89%	F1: 80.7%
--------------------------------------------------------------------------



# Coref results with CDLM model with lemma similarity, high F1 low recall

In [216]:
similarities = get_mention_pair_similarity_lemma(mention_pairs, all_mention_map, relations, working_folder)
#similarities, _ = get_mention_pair_cosinesimilarity(mention_pairs, all_mention_map,vector_map_evt_longbertdoc, relations, working_folder)

In [218]:
# get the similarities of the mention-pairs
# similarities = get_mention_pair_similarity_lemma(mention_pairs, all_mention_map, relations, working_folder)

# get indices
mention_ind_pairs = [(curr_men_to_ind[mp[0]], curr_men_to_ind[mp[1]]) for mp in mention_pairs]
row, col = zip(*mention_ind_pairs)
# print(row, col)

# create similarity matrix from the similarities
n = len(curr_mentions)
similarities = [int(x) for x in similarities ] # to avoid ValueError: row, column, and data arrays must be 1-D
similarity_matrix = csr_matrix((similarities, (row, col)), shape=(n, n)).toarray()

# clustering algorithm and mention cluster map
clusters, labels = connected_components(similarity_matrix)
system_mention_cluster_map = [(men, clus) for men, clus in zip(curr_mentions, labels)]

# generate system key file
system_key_file = working_folder + f'/{men_type}_system.keyfile'
generate_key_file(system_mention_cluster_map, men_type, working_folder, system_key_file)

# evaluate
generate_results(gold_key_file, system_key_file)

BCUB SCORE
====== TOTALS =======
Identification of Mentions: Recall: (847 / 847) 100%	Precision: (847 / 847) 100%	F1: 100%
--------------------------------------------------------------------------
Coreference: Recall: (330.836894933412 / 847) 39.05%	Precision: (586.129012327542 / 847) 69.2%	F1: 49.93%
--------------------------------------------------------------------------

MUC SCORE
====== TOTALS =======
Identification of Mentions: Recall: (847 / 847) 100%	Precision: (847 / 847) 100%	F1: 100%
--------------------------------------------------------------------------
Coreference: Recall: (226 / 576) 39.23%	Precision: (226 / 399) 56.64%	F1: 46.35%
--------------------------------------------------------------------------



# BERT large embeddings and coref results

In [226]:
cross_encoder = FullCrossEncoderSingle(model_bert_large.config , long=True)

loading file /s/chopin/d/proj/ramfis-aida/coreference_and_annotations/coreference/bert_large/vocab.txt
loading file /s/chopin/d/proj/ramfis-aida/coreference_and_annotations/coreference/bert_large/added_tokens.json
loading file /s/chopin/d/proj/ramfis-aida/coreference_and_annotations/coreference/bert_large/special_tokens_map.json
loading file /s/chopin/d/proj/ramfis-aida/coreference_and_annotations/coreference/bert_large/tokenizer_config.json
Adding <m> to the vocabulary
Adding </m> to the vocabulary
loading configuration file /s/chopin/d/proj/ramfis-aida/coreference_and_annotations/coreference/bert_large/config.json
Model config BertConfig {
  "_name_or_path": "bert-large-uncased",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
 

In [227]:
cross_encoder = cross_encoder.to(device)
cross_encoder.model = BertModel.from_pretrained(bert_large_path).to(device)

loading configuration file /s/chopin/d/proj/ramfis-aida/coreference_and_annotations/coreference/bert_large/config.json
Model config BertConfig {
  "_name_or_path": "bert-large-uncased",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30524
}

loading weights file /s/chopin/d/proj/ramfis-aida/coreference_and_annotations/coreference/bert_large/pytorch_model.bin
All model checkpoint weights were used when initializing BertModel.

All the 

In [228]:
parallel_bert_large_model = torch.nn.DataParallel(cross_encoder)
parallel_bert_large_model.module.to(device)

FullCrossEncoderSingle(
  (model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30524, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-12, 

In [229]:
parallel_bert_large_model.eval()
parallel_bert_large_model.module.tokenizer

PreTrainedTokenizer(name_or_path='/s/chopin/d/proj/ramfis-aida/coreference_and_annotations/coreference/bert_large', vocab_size=30522, model_max_len=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]', 'additional_special_tokens': ['<m>', '</m>']})

In [231]:
parallel_bert_large_model.module.vals[0],parallel_bert_large_model.module.vals[1]
 

(30522, 30523)

In [245]:
loc_mentions = []
bert_sentences
batch = [bert_sentences[x] for x in range( len(bert_sentences))]
 
bert_tokens = parallel_bert_large_model.module.tokenizer(batch, pad_to_max_length=True, add_special_tokens=False, truncation =False )
#bert_tokens = parallel_model.module.tokenizer(batch, padding = 'max_length', add_special_tokens=False, truncation =True )
input_ids = torch.tensor(bert_tokens['input_ids'], device=device)
attention_mask = torch.tensor(bert_tokens['attention_mask'], device=device)
m = input_ids.cpu()
k = m == parallel_bert_large_model.module.vals[0]
p = m == parallel_bert_large_model.module.vals[1]
v = (k.int() + p.int()).bool()
nz_indexes = v.nonzero()[:, 1].reshape(m.shape[0], 2)
print(nz_indexes.shape)
#print(nz_indexes[0][1])
loc_mentions.extend(nz_indexes.cpu().numpy())
print(input_ids.shape)
loc_mentions


torch.Size([847, 2])
torch.Size([847, 10946])


[array([116, 118]),
 array([57, 59]),
 array([142, 144]),
 array([454, 456]),
 array([616, 619]),
 array([334, 338]),
 array([695, 698]),
 array([696, 698]),
 array([858, 861]),
 array([895, 897]),
 array([928, 930]),
 array([928, 930]),
 array([952, 954]),
 array([949, 951]),
 array([975, 977]),
 array([975, 977]),
 array([1011, 1013]),
 array([1038, 1040]),
 array([1038, 1040]),
 array([1051, 1053]),
 array([1073, 1075]),
 array([1086, 1089]),
 array([1106, 1108]),
 array([1106, 1111]),
 array([222, 227]),
 array([616, 619]),
 array([518, 521]),
 array([962, 964]),
 array([978, 981]),
 array([1087, 1090]),
 array([244, 246]),
 array([29, 32]),
 array([380, 382]),
 array([223, 225]),
 array([526, 529]),
 array([542, 545]),
 array([559, 562]),
 array([379, 381]),
 array([533, 536]),
 array([547, 549]),
 array([560, 562]),
 array([436, 439]),
 array([30, 32]),
 array([26, 29]),
 array([26, 29]),
 array([57, 59]),
 array([57, 59]),
 array([117, 120]),
 array([117, 120]),
 array([3, 5]),


In [246]:
overflow_idx=[]
[x[1] for x in loc_mentions]
for i,j in enumerate(loc_mentions):
    if j[1]>=512:
        overflow_idx.append([i, j[1] ])
        
        
        
idx = [x[0] for x in overflow_idx]

# res=list(np.array(bert_sentences)[idx])
# len(res)

m_new  = []
for i, j in enumerate(overflow_idx):
    print(j )
    bert_sent = bert_sentences[j[0] ]
    
    #m[j[0]] = m[j[0], j[1]-4096:j[1]] 
    print("m shape",m[j[0],  j[1]-512+1:j[1]+1].shape,j[0] )
    m_new.append(m[j[0],  j[1]-512+1:j[1]+1] )
    print(len(bert_sent.split()))
    
    


[4, 619]
m shape torch.Size([512]) 4
740
[6, 698]
m shape torch.Size([512]) 6
740
[7, 698]
m shape torch.Size([512]) 7
740
[8, 861]
m shape torch.Size([512]) 8
740
[9, 897]
m shape torch.Size([512]) 9
740
[10, 930]
m shape torch.Size([512]) 10
741
[11, 930]
m shape torch.Size([512]) 11
741
[12, 954]
m shape torch.Size([512]) 12
741
[13, 951]
m shape torch.Size([512]) 13
740
[14, 977]
m shape torch.Size([512]) 14
740
[15, 977]
m shape torch.Size([512]) 15
740
[16, 1013]
m shape torch.Size([512]) 16
740
[17, 1040]
m shape torch.Size([512]) 17
740
[18, 1040]
m shape torch.Size([512]) 18
740
[19, 1053]
m shape torch.Size([512]) 19
740
[20, 1075]
m shape torch.Size([512]) 20
740
[21, 1089]
m shape torch.Size([512]) 21
740
[22, 1108]
m shape torch.Size([512]) 22
740
[23, 1111]
m shape torch.Size([512]) 23
740
[25, 619]
m shape torch.Size([512]) 25
740
[26, 521]
m shape torch.Size([512]) 26
740
[27, 964]
m shape torch.Size([512]) 27
741
[28, 981]
m shape torch.Size([512]) 28
741
[29, 1090]
m 

In [247]:
m_new_tensor = torch.stack(m_new) 
 
#torch.nn.functional.pad(m_new_tensor, pad = (0,1), mode='constant', value=1)
m_new_tensor = F.pad(m_new_tensor,pad =(0, m.shape[1]-m_new_tensor.shape[1]), value=1) 
m[idx] = m_new_tensor


In [248]:
m.shape

torch.Size([847, 10946])

In [249]:
k = m == parallel_bert_large_model.module.vals[0]
p = m == parallel_bert_large_model.module.vals[1]

v = (k.int() + p.int()).bool()
nz_indexes = v.nonzero()[:, 1].reshape(m.shape[0], 2)

q = torch.arange(m.shape[1])
q = q.repeat(m.shape[0], 1)

msk_0 = (nz_indexes[:, 0].repeat(m.shape[1], 1).transpose(0, 1)) <= q
msk_1 = (nz_indexes[:, 1].repeat(m.shape[1], 1).transpose(0, 1)) >= q

msk_0_ar = (nz_indexes[:, 0].repeat(m.shape[1], 1).transpose(0, 1)) < q
msk_1_ar = (nz_indexes[:, 1].repeat(m.shape[1], 1).transpose(0, 1)) > q

attention_mask_g = msk_0.int() * msk_1.int()

input_ids = input_ids[:, :512]
# attention_mask = attention_mask[:, :4096]
#attention_mask[:, 0] = 2
#attention_mask[attention_mask_g == 1] = 2
attention_mask = attention_mask[:, :512]
arg1 = msk_0_ar.int() * msk_1_ar.int()
arg_new1 = msk_0.int() * msk_1.int()
arg_new1 = arg_new1[:,:512]
arg1 = arg1[:, :512]
arg1 = arg1.to(device)

In [254]:
input_ids.shape, attention_mask.shape, arg1.shape

(torch.Size([847, 512]), torch.Size([847, 512]), torch.Size([847, 512]))

In [255]:
batch_size = 150
batch = [input_ids[x] for x in range(i, min(i + batch_size, len(bert_sentences)))]
for i in tqdm(range(0, len(bert_sentences), batch_size), desc="generating "):
    print(input_ids[i:i+batch_size].shape)
    
    print(i)

generating : 100%|██████████| 6/6 [00:00<00:00, 36738.43it/s]

torch.Size([150, 512])
0
torch.Size([150, 512])
150
torch.Size([150, 512])
300
torch.Size([150, 512])
450
torch.Size([150, 512])
600
torch.Size([97, 512])
750


In [256]:
all_vectors = []
batch_size = 50
with torch.no_grad():
    for i in tqdm(range(0, len(bert_sentences), batch_size), desc="generating "):
        #men_vectors = parallel_coref_model(input_ids[i:i+batch_size], attention_mask[i:i+batch_size], arg1[i:i+batch_size]).detach().cpu().numpy()
        men_vectors = parallel_bert_large_model(input_ids[i:i+batch_size], attention_mask[i:i+batch_size], arg1[i:i+batch_size]) 
        all_vectors.extend(men_vectors)

generating : 100%|██████████| 17/17 [00:12<00:00,  1.41it/s]


In [258]:
zip_iterator = zip(men_ids, all_vectors)

In [259]:
bert_large_eve_vec_map_longdoc = dict(zip_iterator)

In [262]:
bert_large_eve_vec_map_longdoc['VMIC0015MXG.000003'] 

tensor([-0.0027, -0.7967, -1.0430,  ..., -0.8410,  0.2529,  0.0932],
       device='cuda:0')

In [263]:
bert_large_eve_mention_map_longdoc_file = working_folder + '/cbert_large_evt_vectorn_map_longbertdoc.pkl'
#pickle.dump(bert_large_eve_vec_map_longdoc, open(bert_large_eve_mention_map_longdoc_file, 'wb'))

In [264]:
bert_large_vector_map_evt_longbertdoc = pickle.load(open(bert_large_eve_mention_map_longdoc_file, 'rb'))

In [265]:
bert_large_vector_map_evt_longbertdoc['VMIC0015MXG.000003'] 

tensor([-0.0027, -0.7967, -1.0430,  ..., -0.8410,  0.2529,  0.0932],
       device='cuda:0')

In [270]:

similarities, _ = get_mention_pair_cosinesimilarity(mention_pairs, all_mention_map,bert_large_vector_map_evt_longbertdoc, relations, working_folder)

/s/chopin/d/proj/ramfis-aida/venv/lib/python3.6/site-packages/ipykernel_launcher.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/s/chopin/d/proj/ramfis-aida/venv/lib/python3.6/site-packages/ipykernel_launcher.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [273]:
similarities = np.asarray(similarities)
np.max(similarities), np.min(similarities), np.mean(similarities)
similarities

array([[1.],
       [0.],
       [0.],
       ...,
       [0.],
       [1.],
       [1.]], dtype=float32)

# bert large coref results with mean = 0.39 as cosine sim threshold 

In [274]:
# get the similarities of the mention-pairs
# similarities = get_mention_pair_similarity_lemma(mention_pairs, all_mention_map, relations, working_folder)

# get indices
mention_ind_pairs = [(curr_men_to_ind[mp[0]], curr_men_to_ind[mp[1]]) for mp in mention_pairs]
row, col = zip(*mention_ind_pairs)
# print(row, col)

# create similarity matrix from the similarities
n = len(curr_mentions)
similarities = [int(x) for x in similarities ] # to avoid ValueError: row, column, and data arrays must be 1-D
similarity_matrix = csr_matrix((similarities, (row, col)), shape=(n, n)).toarray()

# clustering algorithm and mention cluster map
clusters, labels = connected_components(similarity_matrix)
system_mention_cluster_map = [(men, clus) for men, clus in zip(curr_mentions, labels)]

# generate system key file
system_key_file = working_folder + f'/{men_type}_system.keyfile'
generate_key_file(system_mention_cluster_map, men_type, working_folder, system_key_file)

# evaluate
generate_results(gold_key_file, system_key_file)

BCUB SCORE
====== TOTALS =======
Identification of Mentions: Recall: (847 / 847) 100%	Precision: (847 / 847) 100%	F1: 100%
--------------------------------------------------------------------------
Coreference: Recall: (844 / 847) 99.64%	Precision: (58.5333491877758 / 847) 6.91%	F1: 12.92%
--------------------------------------------------------------------------

MUC SCORE
====== TOTALS =======
Identification of Mentions: Recall: (847 / 847) 100%	Precision: (847 / 847) 100%	F1: 100%
--------------------------------------------------------------------------
Coreference: Recall: (573 / 576) 99.47%	Precision: (573 / 844) 67.89%	F1: 80.7%
--------------------------------------------------------------------------



# bert large coref results with mean = 0.47 as cosine sim threshold 

In [276]:

similarities, _ = get_mention_pair_cosinesimilarity(mention_pairs, all_mention_map,bert_large_vector_map_evt_longbertdoc, relations, working_folder)

/s/chopin/d/proj/ramfis-aida/venv/lib/python3.6/site-packages/ipykernel_launcher.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/s/chopin/d/proj/ramfis-aida/venv/lib/python3.6/site-packages/ipykernel_launcher.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [277]:
# get the similarities of the mention-pairs
# similarities = get_mention_pair_similarity_lemma(mention_pairs, all_mention_map, relations, working_folder)

# get indices
mention_ind_pairs = [(curr_men_to_ind[mp[0]], curr_men_to_ind[mp[1]]) for mp in mention_pairs]
row, col = zip(*mention_ind_pairs)
# print(row, col)

# create similarity matrix from the similarities
n = len(curr_mentions)
similarities = [int(x) for x in similarities ] # to avoid ValueError: row, column, and data arrays must be 1-D
similarity_matrix = csr_matrix((similarities, (row, col)), shape=(n, n)).toarray()

# clustering algorithm and mention cluster map
clusters, labels = connected_components(similarity_matrix)
system_mention_cluster_map = [(men, clus) for men, clus in zip(curr_mentions, labels)]

# generate system key file
system_key_file = working_folder + f'/{men_type}_system.keyfile'
generate_key_file(system_mention_cluster_map, men_type, working_folder, system_key_file)

# evaluate
generate_results(gold_key_file, system_key_file)

BCUB SCORE
====== TOTALS =======
Identification of Mentions: Recall: (847 / 847) 100%	Precision: (847 / 847) 100%	F1: 100%
--------------------------------------------------------------------------
Coreference: Recall: (834.646008403361 / 847) 98.54%	Precision: (69.1464681153328 / 847) 8.16%	F1: 15.07%
--------------------------------------------------------------------------

MUC SCORE
====== TOTALS =======
Identification of Mentions: Recall: (847 / 847) 100%	Precision: (847 / 847) 100%	F1: 100%
--------------------------------------------------------------------------
Coreference: Recall: (569 / 576) 98.78%	Precision: (569 / 834) 68.22%	F1: 80.7%
--------------------------------------------------------------------------



# bert large coref results with mean = 0.6 as cosine sim threshold 

In [280]:

similarities, _ = get_mention_pair_cosinesimilarity(mention_pairs, all_mention_map,bert_large_vector_map_evt_longbertdoc, relations, working_folder)

/s/chopin/d/proj/ramfis-aida/venv/lib/python3.6/site-packages/ipykernel_launcher.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/s/chopin/d/proj/ramfis-aida/venv/lib/python3.6/site-packages/ipykernel_launcher.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [282]:
# get the similarities of the mention-pairs
# similarities = get_mention_pair_similarity_lemma(mention_pairs, all_mention_map, relations, working_folder)

# get indices
mention_ind_pairs = [(curr_men_to_ind[mp[0]], curr_men_to_ind[mp[1]]) for mp in mention_pairs]
row, col = zip(*mention_ind_pairs)
# print(row, col)

# create similarity matrix from the similarities
n = len(curr_mentions)
similarities = [int(x) for x in similarities ] # to avoid ValueError: row, column, and data arrays must be 1-D
similarity_matrix = csr_matrix((similarities, (row, col)), shape=(n, n)).toarray()

# clustering algorithm and mention cluster map
clusters, labels = connected_components(similarity_matrix)
system_mention_cluster_map = [(men, clus) for men, clus in zip(curr_mentions, labels)]

# generate system key file
system_key_file = working_folder + f'/{men_type}_system.keyfile'
generate_key_file(system_mention_cluster_map, men_type, working_folder, system_key_file)

# evaluate
generate_results(gold_key_file, system_key_file)

BCUB SCORE
====== TOTALS =======
Identification of Mentions: Recall: (847 / 847) 100%	Precision: (847 / 847) 100%	F1: 100%
--------------------------------------------------------------------------
Coreference: Recall: (617.269597590231 / 847) 72.87%	Precision: (214.491595655806 / 847) 25.32%	F1: 37.58%
--------------------------------------------------------------------------

MUC SCORE
====== TOTALS =======
Identification of Mentions: Recall: (847 / 847) 100%	Precision: (847 / 847) 100%	F1: 100%
--------------------------------------------------------------------------
Coreference: Recall: (458 / 576) 79.51%	Precision: (458 / 708) 64.68%	F1: 71.33%
--------------------------------------------------------------------------



# bert large coref results with mean = 0.7 as cosine sim threshold 

In [284]:

similarities, _ = get_mention_pair_cosinesimilarity(mention_pairs, all_mention_map,bert_large_vector_map_evt_longbertdoc, relations, working_folder)

/s/chopin/d/proj/ramfis-aida/venv/lib/python3.6/site-packages/ipykernel_launcher.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/s/chopin/d/proj/ramfis-aida/venv/lib/python3.6/site-packages/ipykernel_launcher.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [285]:
# get the similarities of the mention-pairs
# similarities = get_mention_pair_similarity_lemma(mention_pairs, all_mention_map, relations, working_folder)

# get indices
mention_ind_pairs = [(curr_men_to_ind[mp[0]], curr_men_to_ind[mp[1]]) for mp in mention_pairs]
row, col = zip(*mention_ind_pairs)
# print(row, col)

# create similarity matrix from the similarities
n = len(curr_mentions)
similarities = [int(x) for x in similarities ] # to avoid ValueError: row, column, and data arrays must be 1-D
similarity_matrix = csr_matrix((similarities, (row, col)), shape=(n, n)).toarray()

# clustering algorithm and mention cluster map
clusters, labels = connected_components(similarity_matrix)
system_mention_cluster_map = [(men, clus) for men, clus in zip(curr_mentions, labels)]

# generate system key file
system_key_file = working_folder + f'/{men_type}_system.keyfile'
generate_key_file(system_mention_cluster_map, men_type, working_folder, system_key_file)

# evaluate
generate_results(gold_key_file, system_key_file)

BCUB SCORE
====== TOTALS =======
Identification of Mentions: Recall: (847 / 847) 100%	Precision: (847 / 847) 100%	F1: 100%
--------------------------------------------------------------------------
Coreference: Recall: (375.806594427243 / 847) 44.36%	Precision: (410.370821900879 / 847) 48.44%	F1: 46.31%
--------------------------------------------------------------------------

MUC SCORE
====== TOTALS =======
Identification of Mentions: Recall: (847 / 847) 100%	Precision: (847 / 847) 100%	F1: 100%
--------------------------------------------------------------------------
Coreference: Recall: (298 / 576) 51.73%	Precision: (298 / 542) 54.98%	F1: 53.3%
--------------------------------------------------------------------------

